<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=a124af4eccc2ded43004e9d725f228ff4826322dc61b34c868bf9566582a0b1c
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-24 13:11:27
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  400.00
Current:  1.43 C
-------------------
Today PnL: -52.45 K (-0.37%)
Current PnL: -23.15 L (-15.17%)
CY Booked + Current PnL: -9.26 L (-6.07%)
-------------------
Total profit:  1.55 L
Total loss:  -24.70 L
-------------------
Total Booked + Current PnL: 17.72 L (14.11%)
Total Booked PnL: 40.87 L (32.53%)
Curr Year Booked PnL: 13.89 L (9.74%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 88.56 L (62.08%)
Deployed:  1.26 C
Current:  1.43 C
CAGR/XIRR %: 7.24%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.00,0.24,-14.03,16.43,0.09,14234.0,-14140.0,86637.0,101.07,50.0,M-SC,4.26,253.0,-0.99,0.61,13.82,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,0.28,-8.73,9.59,0.02,15278.0,-15244.0,159315.0,142.75,71.0,M-SC,5.46,234.0,-1.00,1.12,38.58,OX40N,NTT,PAINTS
43,ITC,452.00,-0.67,-1.02,11.58,10.44,22939.0,-2044.0,198094.0,-40.74,45.0,X-LC,0.77,5.0,-0.09,1.39,4.49,X40,NTT,FMCG
53,NESTLEIND,1377.00,-0.23,12.64,7.77,21.40,23231.0,33556.0,298982.0,4.55,60.0,X-LC,7.93,12.0,1.44,2.10,20.13,XY25,NTT,FMCG
35,ICICIGI,2252.93,-0.26,6.64,12.71,20.19,23628.0,11578.0,185898.0,-16.49,50.0,X-MC,7.63,68.0,0.49,1.30,22.76,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
34,HONAUT,58357.33,2.64,-13.96,66.57,43.32,93291.0,-22732.0,140140.0,-26.09,40.0,X-SC,1.94,90.0,-0.24,0.98,7.60,X40N,ATH,ELECTRICAL
70,SURYODAY,216.00,2.16,-17.93,49.91,23.03,73354.0,-32099.0,146972.0,60.09,52.0,H-SC,11.23,167.0,-0.44,1.03,45.83,XR,NTT,BANKS
29,HAPPSTMNDS,1480.71,1.64,-36.95,193.18,84.85,156105.0,-47357.0,80808.0,-25.05,48.0,H-SC,14.25,147.0,-0.30,0.57,3.71,AR,ATH,IT
26,GILLETTE,11206.78,1.59,-0.65,33.28,32.42,83950.0,-1641.0,252255.0,-18.36,36.0,X-SC,6.64,85.0,-0.02,1.77,14.35,X40,ATH,FMCG
85,WIPRO,420.00,1.48,1.76,69.29,72.27,119477.0,2982.0,172430.0,-12.89,58.0,M-LC,4.60,99.0,0.02,1.21,8.04,XR,NTT,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,VALIANTORG,838.00,-4.06,-46.45,225.18,74.15,197298.0,-75987.0,87618.0,-257.83,25.0,H-SC,14.61,149.0,-0.39,0.61,12.31,XR,NTT,CHEMICALS
27,GLAXO,3466.20,-3.51,-3.24,43.81,39.15,84476.0,-6460.0,192824.0,-22.38,26.0,X-MC,9.12,60.0,-0.08,1.35,24.82,X40,ATH,PHARMA
72,TANLA,1943.92,-3.23,-36.83,247.78,119.69,423800.0,-99722.0,171039.0,-36.34,26.0,H-SC,12.01,156.0,-0.24,1.20,35.12,AR,ATH,IT
6,ASIANTILES,137.00,-3.07,-15.44,114.83,81.67,91095.0,-14480.0,79330.0,7127.78,44.0,L-SC,18.44,271.0,-0.16,0.56,53.63,XR,NTT,CERAMICS
58,RAJOOENG,143.10,-2.95,-29.70,98.58,39.61,71037.0,-30440.0,72060.0,-52.38,20.0,H-SC,19.27,135.0,-0.43,0.51,0.00,AR,ATH,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-1.14,-0.52,107.79,106.72,132258.0,-636.0,122700.0,-50.82,33.0,H-SC,4.07,139.0,-0.0,0.86,22.74,AR,ATH,MISC


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-1.48,-5.40,119.78,107.91,167297.0,-7974.0,139670.0,-22.98,32.0,M-SC,10.37,216.0,-0.05,0.98,0.00,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.28,-8.73,9.59,0.02,15278.0,-15244.0,159315.0,142.75,71.0,M-SC,5.46,234.0,-1.00,1.12,38.58,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,0.24,-14.03,16.43,0.09,14234.0,-14140.0,86637.0,101.07,50.0,M-SC,4.26,253.0,-0.99,0.61,13.82,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-0.35,-22.58,46.56,13.47,97209.0,-60885.0,208782.0,-68.12,27.0,H-SC,1.52,158.0,-0.63,1.46,6.16,XY24,NTT,PAINTS
73,TATAELXSI,9161.00,0.75,-22.74,70.90,32.03,72212.0,-29985.0,101850.0,-16.37,50.0,H-SC,6.62,148.0,-0.42,0.71,14.08,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.0,1.48,1.76,69.29,72.27,119477.0,2982.0,172430.0,-12.89,58.0,M-LC,4.6,99.0,0.02,1.21,8.04,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,1.48,1.76,69.29,72.27,119477.0,2982.0,172430.0,-12.89,58.0,M-LC,4.60,99.0,0.02,1.21,8.04,XR,NTT,IT
38,INDIAMART,4810.62,-1.14,-0.52,107.79,106.72,132258.0,-636.0,122700.0,-50.82,33.0,H-SC,4.07,139.0,-0.00,0.86,22.74,AR,ATH,MISC
25,FINCABLES,1641.55,-1.48,-5.40,119.78,107.91,167297.0,-7974.0,139670.0,-22.98,32.0,M-SC,10.37,216.0,-0.05,0.98,0.00,OX40N,ATH,CABLES
86,ZYDUSLIFE,1286.17,0.27,-2.27,38.78,35.63,79430.0,-4751.0,204823.0,-15.85,34.0,H-MC,4.52,119.0,-0.06,1.44,14.03,AR,ATH,PHARMA
84,WHIRLPOOL,2270.00,-0.45,-5.83,93.19,81.92,100738.0,-6698.0,108100.0,-44.81,31.0,M-SC,5.30,236.0,-0.07,0.76,27.97,XR,NTT,DURABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
46,JSWINFRA,342.0,-0.87,-9.86,27.83,15.23,50260.0,-19751.0,180596.0,-22.98,23.0,X-MC,6.51,66.0,-0.39,1.27,19.21,X40N,NTT,REALTY
76,TMPV,600.0,-0.58,-14.66,66.60,42.18,157108.0,-40512.0,235898.0,-23.08,23.0,X-LC,0.39,3.0,-0.26,1.65,0.10,XY24,NTT,AUTO
11,BATAINDIA,2096.0,0.35,-35.24,108.79,35.20,90648.0,-45346.0,83324.0,9.31,24.0,X-SC,12.62,92.0,-0.50,0.58,0.35,X40,NTT,FOOTWEAR
27,GLAXO,3466.2,-3.51,-3.24,43.81,39.15,84476.0,-6460.0,192824.0,-22.38,26.0,X-MC,9.12,60.0,-0.08,1.35,24.82,X40,ATH,PHARMA
60,RELAXO,1176.0,-0.88,-47.57,190.48,52.31,144978.0,-69048.0,76112.0,-44.15,28.0,X-SC,3.79,91.0,-0.48,0.53,1.88,X40N,NTT,FOOTWEAR


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,600.0,-0.58,-14.66,66.60,42.18,157108.0,-40512.0,235898.0,-23.08,23.0,X-LC,0.39,3.0,-0.26,1.65,0.10,XY24,NTT,AUTO
43,ITC,452.0,-0.67,-1.02,11.58,10.44,22939.0,-2044.0,198094.0,-40.74,45.0,X-LC,0.77,5.0,-0.09,1.39,4.49,X40,NTT,FMCG
36,ICICIPRULI,790.0,0.00,1.28,29.32,30.97,52481.0,2256.0,178994.0,-22.52,49.0,X-MC,1.54,75.0,0.04,1.25,14.00,X40,ATH,INSURANCE
3,ACC,3906.0,-0.46,-23.45,114.43,64.15,208446.0,-55791.0,182160.0,-55.44,41.0,X-SC,1.90,82.0,-0.27,1.28,2.18,XY24,BTT,CEMENT
66,SIEMENS,4671.5,0.11,-14.64,47.04,25.51,74723.0,-27245.0,158850.0,-2.45,54.0,H-LC,1.93,49.0,-0.36,1.11,17.06,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,600.00,-0.58,-14.66,66.60,42.18,157108.0,-40512.0,235898.0,-23.08,23.0,X-LC,0.39,3.0,-0.26,1.65,0.10,XY24,NTT,AUTO
11,BATAINDIA,2096.00,0.35,-35.24,108.79,35.20,90648.0,-45346.0,83324.0,9.31,24.0,X-SC,12.62,92.0,-0.50,0.58,0.35,X40,NTT,FOOTWEAR
9,BAJAJHFL,181.50,0.49,-13.26,71.15,48.45,123746.0,-26584.0,173922.0,-19.44,34.0,X-MC,7.45,64.0,-0.21,1.22,0.49,X40N,ATH,FINANCE
54,PAGEIND,51605.08,-0.14,-5.14,32.90,26.07,51100.0,-8420.0,155320.0,-30.18,35.0,X-MC,8.87,55.0,-0.16,1.09,0.69,X40,ATH,APPARELS
55,PGHH,17907.65,-0.10,-4.12,39.50,33.76,76059.0,-8265.0,192555.0,-32.38,34.0,X-MC,3.19,57.0,-0.11,1.35,0.87,X40,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,1.33,-28.36,99.39,42.84,46286.0,-18436.0,46570.0,-53.17,39.0,X-SC,37.24,83.0,-0.40,0.33,1.33,XY24,NTT,MISC
60,RELAXO,1176.00,-0.88,-47.57,190.48,52.31,144978.0,-69048.0,76112.0,-44.15,28.0,X-SC,3.79,91.0,-0.48,0.53,1.88,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,0.35,-35.24,108.79,35.20,90648.0,-45346.0,83324.0,9.31,24.0,X-SC,12.62,92.0,-0.50,0.58,0.35,X40,NTT,FOOTWEAR
52,MEDANTA,1486.00,0.25,2.70,20.81,24.08,27132.0,3430.0,130380.0,-5.65,44.0,X-SC,4.75,89.0,0.13,0.91,22.85,XY24,NTT,HEALTHCARE
22,DIXON,18931.72,-1.09,-4.94,27.90,21.58,37168.0,-6921.0,133218.0,-52.09,33.0,X-MC,7.02,56.0,-0.19,0.93,17.94,X40N,ATH,IT


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.96,0.44,-14.02,38.73,19.28,115207.0,-48493.0,297463.0,-24.92,65.0,X-LC,7.63,1.0,-0.42,2.09,9.97,X40,ATH,IT
41,INFY,2275.00,1.22,6.46,45.48,54.87,151489.0,20201.0,333089.0,-17.09,63.0,X-LC,3.15,2.0,0.13,2.34,15.27,X40,BTT,IT
76,TMPV,600.00,-0.58,-14.66,66.60,42.18,157108.0,-40512.0,235898.0,-23.08,23.0,X-LC,0.39,3.0,-0.26,1.65,0.10,XY24,NTT,AUTO
82,VBL,671.64,-0.01,-9.58,50.05,35.67,142927.0,-30273.0,285569.0,-20.30,41.0,X-LC,6.05,4.0,-0.21,2.00,2.94,X40N,ATH,FMCG
43,ITC,452.00,-0.67,-1.02,11.58,10.44,22939.0,-2044.0,198094.0,-40.74,45.0,X-LC,0.77,5.0,-0.09,1.39,4.49,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,-3.07,-15.44,114.83,81.67,91095.0,-14480.0,79330.0,7127.78,44.0,L-SC,18.44,271.0,-0.16,0.56,53.63,XR,NTT,CERAMICS
68,SONACOMS,806.63,1.34,-11.79,58.19,39.54,51924.0,-11927.0,89232.0,-31.03,69.0,M-SC,6.85,220.0,-0.23,0.63,25.83,AR,ATH,AUTO
51,MASFIN,398.61,-2.05,-4.91,28.36,22.05,26422.0,-4815.0,93165.0,-17.80,51.0,H-SC,7.22,164.0,-0.18,0.65,35.42,XR,ATH,FINANCE
70,SURYODAY,216.00,2.16,-17.93,49.91,23.03,73354.0,-32099.0,146972.0,60.09,52.0,H-SC,11.23,167.0,-0.44,1.03,45.83,XR,NTT,BANKS
39,INDIGOPNTS,1408.00,0.28,-8.73,9.59,0.02,15278.0,-15244.0,159315.0,142.75,71.0,M-SC,5.46,234.0,-1.00,1.12,38.58,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,0.28,-8.73,9.59,0.02,15278.0,-15244.0,159315.0,142.75,71.0,M-SC,5.46,234.0,-1.00,1.12,38.58,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,0.11,0.66,20.16,20.95,51090.0,1654.0,253422.0,-0.35,76.0,X-LC,12.74,31.0,0.03,1.78,36.53,X40,ATH,PAINTS
40,INDUSINDBK,1800.00,0.14,-37.42,112.34,32.89,54282.0,-28888.0,48319.0,-696.43,64.0,L-MC,5.62,259.0,-0.53,0.34,33.07,XR,NTT,BANKS
0,5PAISA,593.00,-2.07,-36.08,76.62,12.90,107017.0,-78824.0,139672.0,105.53,56.0,H-SC,14.56,161.0,-0.74,0.98,16.14,OX40N,NTT,FINANCE
50,LTIM,7201.88,0.27,8.63,21.20,31.66,56687.0,21239.0,267390.0,4.56,63.0,H-LC,12.49,43.0,0.37,1.87,48.36,X200,ATH,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.84
1,25,44.74
2,50,75.51


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.54
MC    29.26
LC    25.22
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.88
X40      21.23
X40N     11.89
XR        9.64
XY25      9.10
AR        8.97
OX40N     7.68
X200      1.87
SR        0.98
MH        0.78
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.90
X-MC    22.83
X-LC    20.06
M-SC    12.04
X-SC     8.21
H-MC     4.72
H-LC     2.98
L-SC     1.39
M-MC     1.37
M-LC     1.21
L-LC     0.97
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.81,-4.96,38.90
IT,13.86,-14.38,73.42
FINANCE,9.42,-15.35,64.10
MISC,7.42,-22.83,73.16
PAINTS,6.01,-7.76,23.76
ELECTRICAL,5.91,-11.19,51.91
INSURANCE,4.40,-0.61,36.07
PHARMA,3.83,-2.36,35.12
AUTO,3.36,-19.91,71.33


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3060938.0,21
AR,1294240.0,10
XR,1286898.0,13
X40,1010811.0,14
X40N,752626.0,9
OX40N,696181.0,10
XY25,343555.0,6
SR,280278.0,2
MH,73572.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3568957.0,25
M-SC,1344524.0,15
X-MC,1224140.0,16
X-LC,903252.0,11
X-SC,748266.0,8
H-MC,406690.0,3
L-SC,263673.0,3
H-LC,131410.0,2
M-LC,119477.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1223669.0      6
           AR         880177.0      5
           XR         788057.0      7
M-SC       XY24       787125.0      6
X-MC       X40        448854.0      7
X-LC       X40        387359.0      5
X-MC       XY24       348149.0      3
H-SC       OX40N      323204.0      4
X-SC       X40N       291804.0      3
M-SC       OX40N      284518.0      5
X-SC       XY24       281864.0      3
H-SC       SR         280278.0      2
X-MC       X40N       247477.0      4
X-LC       XY24       229768.0      2
H-MC       AR         216327.0      2
X-LC       X40N       213345.0      2
H-MC       XY24       190363.0      1
X-MC       XY25       179660.0      2
L-SC       XR         175214.0      2
X-SC       X40        174598.0      2
M-SC       XR         149868.0      2
           AR         123013.0      2
M-LC       XR         119477.0      1
L-SC       OX40N       88459.0      1
H-LC       AR          74723.0      1
H-SC       MH          73572.0      1
X-LC       XY25        72780.0      2
H-LC       X200        56687.0      1
L-MC       XR          54282.0      1
M-MC       XY25        49889.0      1
L-LC       XY25        41226.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 53.0 seconds
